In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
%matplotlib inline

In [43]:
df = pd.read_csv('pdf_links_main.csv', index_col = 0)
df.head()

,Name,Date
0,lsd_12_01_23-03-1998.pdf,23-Mar-1998
1,lsd_12_01_24-03-1998.pdf,24-Mar-1998
2,lsd_12_01_25-03-1998.pdf,25-Mar-1998
3,lsd_12_01_26-03-1998.pdf,26-Mar-1998
4,lsd_12_01_27-03-1998.pdf,27-Mar-1998


In [77]:
df.index[df['Name']=='lsd_12_01_27-03-1998.pdf'].tolist()

[4]

In [33]:
from nltk.corpus import stopwords
#set(stopwords.words('english'))

In [34]:
import os
file_list = os.listdir('text_files')

In [40]:
file_list[0]

'01-03-2005_14.txt'

In [48]:
def clean_text(text):
    text = text.replace('\n', '')
    text = re.sub('\(.*?\)', '', text) #Replace all words enclosed in parentheses with empty string
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text) # Remove extra punctuations with spaces
    text = " ".join(text.split()) #Remove extra spaces
    return text

string = 'The Lok Sabha met at Eleven of the Clock(MR. SPEAKER in the Chair)'
clean_text(string)

'The Lok Sabha met at Eleven of the Clock'

In [97]:
def process_pdf(file):
    f = open(file, 'r', encoding='utf-8')
    data = {}
    text = f.read()
    index = 'LOK SABHA DEBATES'
    last = 'OBSERVATION BY THE SPEAKER NOTICE OF MOTION FOR ADJOURNMENT'
    text = clean_text(text)
    # Every file had the first few pages as noise. The speeches start after the first occurence of the index string
    text = text[text.find(index) + len(index) : ]
    #Every session is started by a speaker
    text = text[text.find('MR SPEAKER') : ]
    text = text[:text.find(last)]
    regex = '[A-Z]+[A-Z]+(?:\s[A-Z]+)*' #Regex expression to extract the names of all the speakers in the session.
    # The speakers name are all in uppercase letters in every PDF file.
    
    l = [(m.start(0), m.end(0)) for m in re.finditer(regex, text)]

    #To create a dictionary of all the speeches of the session
    for i in range(len(l)-1):
        name = text[l[i][0]:l[i][1]-1]
        temp = text[l[i][0]:l[i][1]] + text[l[i][1]:l[i+1][0]]
        temp = temp.replace(name, '')
        data[i] = {'name':name, 'text':temp}
        print(data[i])

    return data

In [98]:
data = process_pdf('text_files/' + file_list[0])

{'name': 'MR SPEAKER ', 'text': 'I have received the notice for Adjournment Motion from the hon Leader of the Opposition I will hear him after the Question Hour Sir I have already given a notice for suspension of Question Hour '}
{'name': 'MR SPEAKER ', 'text': 'Okay I am prepared to hear him on the admissibility of the Adjournment Motion '}
{'name': 'MR SPEAKER ', 'text': 'Please let us have some order I have permitted him to speak When I have permitted him to speak you should not interrupt '}
{'name': 'MR SPEAKER ', 'text': 'Let us have some method of functioning I have permitted him Please do not interrupt '}
{'name': 'MR SPEAKER ', 'text': 'This is an important matter Let us discuss it in a manner which is consistent with the dignity of the House 1967 '}
{'name': 'SHRI MADHUSUDAN MISTRY ', 'text': 'Sir they are making a statement He should speak on admissibility '}
{'name': 'MR SPEAKER ', 'text': 'I will try to answer '}
{'name': 'MR SPEAKER ', 'text': 'Shri Advaniji not on merits 

In [21]:
for file in file_list[:20]:
    path = 'text_files/' + file
    f = open(path, 'r', encoding='utf-8')
    text = f.read()
    text = clean_text(text)
    f.close()
    f = open('clean_text_files/' + file, 'w', encoding='utf-8')
    f.write(text)
    f.close()